<a href="https://colab.research.google.com/github/sjeom6/Colab_Repo/blob/main/langchain_240728_IMG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangChain OpenAI 테스트

In [37]:
!pip install openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.2 MB/s eta 0:00:00


In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = ""

client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [89]:
# 파일 업로드
file = client.files.create(
    file = open("./ISIN_TEST.csv", "rb"),
    purpose = "assistants"
)


In [90]:
# 파일 업로드
file2 = client.files.create(
    file = open("./NASDAQ_CODE_1.csv", "rb"),
    purpose = "assistants"
)


In [91]:
# code_interpreter 사용
assistant = client.beta.assistants.create(
    model="gpt-4o",
    name = "데이터 매칭",
    instructions="각각의 파일(확장자:csv)을 열어 데이터를 비교 해줘 그래고 영어와 한글과 영어로된 이름들을 서로 매칭하여 질문에 해당하는 코드값으로 전달해주면 됩니다. name 비교 시 서로같은 언어로 변경해서 처리하세요.",
    tools=[{"type":"code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [file.id, file2.id]
      }
    }
  )


In [92]:
# 스레드 생성
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_pURUkb55UjOuagITpPzHL2hv', created_at=1722149470, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [93]:
# 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="US0378331005 값을 NAME을 매칭하여 CODE 값을 알려줘"
)
print(message)

Message(id='msg_WTXybtYVilO13LLHJ6STRmQx', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='US0378331005 값을 NAME을 매칭하여 CODE 값을 알려줘'), type='text')], created_at=1722149472, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_pURUkb55UjOuagITpPzHL2hv')


In [94]:
run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
      tools=[{"type":"code_interpreter"}, {"type":"file_search"}]
    )
print(run)

Run(id='run_bZu22DewAES03GVCdAlFcifQ', assistant_id='asst_CoKNVTdmL5hCDaUNKQ2BYsxH', cancelled_at=None, completed_at=None, created_at=1722149475, expires_at=1722150075, failed_at=None, incomplete_details=None, instructions='각각의 파일(확장자:csv)을 열어 데이터를 비교 해줘 그래고 영어와 한글과 영어로된 이름들을 서로 매칭하여 질문에 해당하는 코드값으로 전달해주면 됩니다. name 비교 시 서로같은 언어로 변경해서 처리하세요.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_pURUkb55UjOuagITpPzHL2hv', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [96]:
# 처리결과 확인m
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    run_id=run.id
)

print(messages)

for message in messages:
  if (message.role == "assistant"):
    print(f"어시스턴트: \n{message.content[0].text.value}")
  elif (message.role == "user"):
    print(f"사용자: \n{message.content[0].text.value}")
  else:
    print(f"오류: \n{message.content[0].text.value}")


SyncCursorPage[Message](data=[Message(id='msg_1fpcwBup2R1pZEj40das1kVl', assistant_id='asst_CoKNVTdmL5hCDaUNKQ2BYsxH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='"US0378331005"에 해당하는 CODE 값은 **AAPL**입니다. \n\n이 값은 두 파일의 NAME 열을 비교하여 매칭하였으며, \'US0378331005\'는 한국어 이름 \'애플\'과 영문 이름 \'Apple\'로 매칭되었습니다.'), type='text')], created_at=1722149484, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_bZu22DewAES03GVCdAlFcifQ', status=None, thread_id='thread_pURUkb55UjOuagITpPzHL2hv'), Message(id='msg_qwHBuGZygYDk91EWipUmk2wO', assistant_id='asst_CoKNVTdmL5hCDaUNKQ2BYsxH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='먼저, 각 파일의 구조가 다음과 같음을 확인하였습니다:\n\n- 파일 1 (`file-EXMZqlafPFbBgltFwjWgwVxI`):  \n  ``` \n  CODE    |     NAME\n  -------------------\n  AAPL    |    Apple\n  MSFT    |    Microsoft\n  NVDA    |    NaN\n  ```\n\n- 파일 2 (`file-Xth3

In [98]:
# 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="US67066G1040 값을 알려줘"
)
print(message)

Message(id='msg_JH7IYORftIbTgnNwNUv5uTQP', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='US67066G1040 값을 알려줘'), type='text')], created_at=1722149529, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_pURUkb55UjOuagITpPzHL2hv')


In [99]:
run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
      tools=[{"type":"code_interpreter"}, {"type":"file_search"}]
    )
print(run)




Run(id='run_yCzIxI1cg3X2MwLNXiang0P4', assistant_id='asst_CoKNVTdmL5hCDaUNKQ2BYsxH', cancelled_at=None, completed_at=None, created_at=1722149532, expires_at=1722150132, failed_at=None, incomplete_details=None, instructions='각각의 파일(확장자:csv)을 열어 데이터를 비교 해줘 그래고 영어와 한글과 영어로된 이름들을 서로 매칭하여 질문에 해당하는 코드값으로 전달해주면 됩니다. name 비교 시 서로같은 언어로 변경해서 처리하세요.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_pURUkb55UjOuagITpPzHL2hv', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [100]:
# 처리결과 확인m
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    run_id=run.id
)

print(messages)

for message in messages:
  if (message.role == "assistant"):
    print(f"어시스턴트: \n{message.content[0].text.value}")
  elif (message.role == "user"):
    print(f"사용자: \n{message.content[0].text.value}")
  else:
        print(f"오류: \n{message.content[0].text.value}")

SyncCursorPage[Message](data=[Message(id='msg_2GcpbexGkxdrObW9rxfCSYES', assistant_id='asst_CoKNVTdmL5hCDaUNKQ2BYsxH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='현재 제공된 데이터로는 "US67066G1040"에 해당하는 CODE 값을 찾을 수 없습니다. \n\n원인으로는 파일 1에 "NVIDIA"의 CODE 값이 존재하지 않기 때문입니다("NVDA"의 이름이 NaN으로 표시됨).\n\n이를 확인하기 위해 파일 1의 데이터를 직접 확인해보겠습니다.'), type='text')], created_at=1722149535, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_yCzIxI1cg3X2MwLNXiang0P4', status=None, thread_id='thread_pURUkb55UjOuagITpPzHL2hv'), Message(id='msg_3SXD15fKbYMlEvupIIq94bre', assistant_id='asst_CoKNVTdmL5hCDaUNKQ2BYsxH', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='같은 과정을 사용하여 "US67066G1040"에 해당하는 CODE 값을 확인하겠습니다. 이를 위해 파일을 다시 비교하고 매칭 작업을 수행하겠습니다.'), type='text')], created_at=1722149533, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.mes

어시스턴트: 
먼저 두 파일을 열고 내용을 비교하여 '애플'에 해당하는 코드를 찾아보겠습니다. 

```python
import pandas as pd

# Load the first file
file1_path = '/mnt/data/file-lKllooDTkoByYnf8CNQuGuXM'
file1 = pd.read_csv(file1_path)

# Load the second file
file2_path = '/mnt/data/file-tLDhuie5wo46TgObSGxmFteI'
file2 = pd.read_csv(file2_path)

# Display the first few rows of each file to understand their structure
file1_head = file1.head()
file2_head = file2.head()

file1_head, file2_head
```

이 코드를 사용하여 두 파일의 구조를 파악한 후, '애플'이라는 이름을 검색하여 해당하는 코드를 찾아보겠습니다.


In [16]:
# 메시지 생성 2
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="CODE 값을 알려줘"
)
print(message)

Message(id='msg_ZU8vpzM3jfPRJqDtjcmLdScD', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='CODE 값을 알려줘'), type='text')], created_at=1722145317, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VVpDU1Je55NOQYDecd6EZWGD')


In [19]:
run = client.beta.threads.runs.create(
      thread_id=thread.id,
      assistant_id=assistant.id,
      tools=[{"type":"code_interpreter"}, {"type":"file_search"}]
    )
print(run)

Run(id='run_XrcIQD7kVVeZJKnBbVteeRoZ', assistant_id='asst_ekLJZiBB89BOcMELevv6KWEr', cancelled_at=None, completed_at=None, created_at=1722145353, expires_at=1722145953, failed_at=None, incomplete_details=None, instructions='파일들을 열어 데이터를 비교 영어와 한글로된 이름들을 서로 매칭하여 질문에 해당하는 코드값으로 전달해주면 됩니다.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_VVpDU1Je55NOQYDecd6EZWGD', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter'), FileSearchTool(type='file_search', file_search=None)], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})


In [28]:
# 처리결과 확인m
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    run_id=run.id
)

print(messages)

SyncCursorPage[Message](data=[], object='list', first_id=None, last_id=None, has_more=False)


In [26]:
for message in messages:
  if (message.role == "assistant"):
    print(f"어시스턴트: \n{message.content[0].text.value}")
  elif (message.role == "user"):
    print(f"사용자: \n{message.content[0].text.value}")
  else:
    print(f"오류: \n{message.content[0].text.value}")

어시스턴트: 
파일 내용을 확인한 결과, 두 파일 모두 바이너리 형식으로 보입니다. 이는 두 파일이 CSV 형식이 아닌 다른 형식, 특히 Excel 파일일 가능성이 높습니다. 따라서 이 파일들을 `pd.read_excel`을 사용하여 불러오겠습니다.

```python
# Try loading the files as Excel files
file1 = pd.read_excel(file1_path)
file2 = pd.read_excel(file2_path)

# Display the first few rows of each file to understand their structure
file1_head = file1.head()
file2_head = file2.head()

# Inspect the columns of each file
file1_columns = file1.columns
file2_columns = file2.columns

# Search for '애플' in both files
apple_code_file1 = file1[file1.apply(lambda row: row.astype(str).str.contains('애플').any(), axis=1)]
apple_code_file2 = file2[file2.apply(lambda row: row.astype(str).str.contains('애플').any(), axis=1)]

file1_head, file2_head, file1_columns, file2_columns, apple_code_file1, apple_code_file2
```

다시 시도해 보겠습니다.
어시스턴트: 
이번에는 파서 오류가 발생했습니다. 이 경우 파일의 형식이 CSV가 아니거나, 읽어들이는 방법이 잘못되었을 수 있습니다. 파일을 텍스트 형식으로 먼저 확인한 후, 구체적인 형식을 파악해보겠습니다.

파일들을 텍스트로 읽어보겠습니다.

```python
# Try reading the files as pla

NotFoundError: Error code: 404 - {'error': {'message': "No message found with id 'msg_ojGw5M0uWjRDdcXigbqxIBhO'.", 'type': 'invalid_request_error', 'param': 'after', 'code': None}}